сначала займемся парсингом

In [1]:
from bs4 import BeautifulSoup
import re
import requests
from pprint import pprint

In [2]:
pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=449c93e178aff3565cfef9247dacb0da77101c5f93f0ce0d79432f91d17ebbf6
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [3]:
session = requests.session()

In [4]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [29]:
import re 

In [62]:
from IPython.core.debugger import bdb
def get_nth_page(i,bg,bb):
    # скачиваем
    url = f'https://www.kinopoisk.ru/reviews/type/comment/period/month/page/{i}/#list'
    request = requests.get(url)
    if request.status_code == 200: 
      req = session.get(url, headers={'User-Agent': ua.random})
      page = req.text
      soup = BeautifulSoup(page, 'html.parser')
      good=soup.find_all('div',attrs={'class':'response good'})
      for smth in good: 
        good_words=smth.find_all('p')
        for smth in good_words:
          g=smth.text
          g=g.replace('\xa0',' ')
          g=g.replace('\n\r\n',' ')
          bg.append(g)
      bad=soup.find_all('div',attrs={'class':'response bad'})
      for smth in bad: 
        bad_words=smth.find_all('p')
        for smth in bad_words:
          b=smth.text
          b=b.replace('\xa0',' ')
          b=b.replace('\n\r\n',' ')
          bb.append(b)

In [63]:
bag_good=[]
bag_bad=[]

In [64]:
for i in range(1,31):
  get_nth_page(i,bag_good,bag_bad)

In [67]:
print(len(bag_good))
print(len(bag_bad))

953
254


In [102]:
print(bag_bad[18])

"Блондинка» Эндрю Доминика оказалась настолько ущербной картиной, что я впервые сел писать рецензию, не досмотрев фильм. Только вдумайтесь — ради полноценного отзыва я досмотрел такое убожество, как «Крым», я переборол свое желание свалить с киносеанса четвертой «Матрицы», я закатывал глаза, но продолжал смотреть «Эйфорию», и чтобы написать рецензию к «Необратимости», мне пришлось стиснуть зубы и терпеть. Вот тут не надо быть по образованию проктологом, чтобы понять, что с «Блондинкой» ситуация полная ж… есть. Не помню контекст, но кто-то из моих знакомых говорил, что ругать фильм гораздо тяжелее, чем хвалить. И насколько же мне трудно говорить про «Блондинку». Фильм получился плох не за какие-то ужасные отдельные элементы, а потому что он представляет сотканное белыми нитками нечто, что должно было позиционировать собой историю культовой американской актрисы и секс-символа Мэрилин Монро. Не буду врать — я ждал этот фильм. Новости о том, что фильм расскажет историю Мэрилин Монро, что г

In [83]:
print(bag_good[952])


прямая ссылка
+ комментарийдобавить комментарий



In [124]:
good_final=[]
bad_final=[]

In [125]:
n=3
while n<953:
  good_final.append(bag_good[n])
  n+=5

In [126]:
bank_of_checks=good_final[31:41]
good_final=good_final[:31]

In [127]:
n=3
while n<253:
  bad_final.append(bag_bad[n])
  n+=5

In [128]:
bad_final=bad_final[:31]

До этого момента мы спарсили все данные, получив в итоге два списка, в одном из которых хранятся тексты всех положительных отзывов на кино за месяц, а в другом -- отрицательных. Вся возня с n+5 была нужна, чтобы оставить именно тексты отзывов, убрав из списков никнейм автора, название фильма и тд (просто они не очень удобно парсятся на кинопоиске) + из-за этого пришлось оставить мало отзывов (31 положительный и 31 негативный)

Теперь переходим к обработке текстов отзыва:

In [90]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 8.2 MB 3.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ec6409993dc792da383278779e5b6a22c84880eabc3e41555da42359e9a43afd
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [145]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [152]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [92]:
morph = MorphAnalyzer()

In [138]:
text_good=''

In [139]:
for smth in good_final: 
  text_good=text_good+smth+' '

In [140]:
text_bad=''

In [141]:
for smth in bad_final: 
  text_bad=text_bad+smth+' '

In [157]:
tokens_good = [w.lower() for w in word_tokenize(text_good)]

In [158]:
from nltk.corpus import stopwords
sw = stopwords.words('russian')

In [159]:
filtered_good = [w for w in tokens_good if w not in sw]

In [160]:
good_last=[]
import string
punct = set(string.punctuation)
for smth in filtered_good:
  if smth not in punct:
    good_last.append(smth)

In [161]:
good_lemmas = []
for smth in good_last:
  good_lemmas.append(morph.parse(smth)[0].normal_form)

In [162]:
tokens_bad = [w.lower() for w in word_tokenize(text_bad)]

In [163]:
filtered_bad = [w for w in tokens_bad if w not in sw]

In [164]:
bad_last=[]
for smth in filtered_bad:
  if smth not in punct:
    bad_last.append(smth)

In [165]:
bad_lemmas = []
for smth in bad_last:
  bad_lemmas.append(morph.parse(smth)[0].normal_form)

Мы превратили набор плохих отзывов и набор хороших отзывов в два мешка лемм, теперь немного упорядочим их

In [166]:
dictgood={}
for smth in good_lemmas:
  if smth in dictgood:
    dictgood[smth]+=1
  else:
    dictgood[smth]=1

In [167]:
dictbad={}
for smth in bad_lemmas:
  if smth in dictgood:
    del dictgood[smth]
  else:
    if smth in dictbad:
      dictbad[smth]+=1
    else:
      dictbad[smth]=1

In [169]:
goodfiltred=[]
badfiltred=[]
for smth in dictgood:
  if dictgood[smth]>2:
    goodfiltred.append(smth)
for smth in dictbad:
  if dictbad[smth]>2:
    badfiltred.append(smth)

таким образом, получаем два списка лемм, отфильтрованных с учетом частотности. Теперь лемматизируем оставшиеся отзывы (взятые для проверки) и создадим функцию, которая будет определять тональность отзыва.

In [178]:
checks=[]
for smth in bank_of_checks:
  tokens_check= [w.lower() for w in word_tokenize(smth)]
  filtered_check = [w for w in tokens_check if w not in sw]
  last_check = [w for w in filtered_check if w not in punct]
  lemme_check=[morph.parse(w)[0].normal_form for w in last_check]
  checks.append(lemme_check)

In [214]:
def tonalizator(rate):
  ton=0
  for word in rate:
    if word in goodfiltred:
      ton+=1
    elif word in badfiltred:
      ton=ton-1
  if ton>=0:
    return(1)
  else:
    return(0)

также посчитаем accuracy

In [187]:
pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=05b765bd927b736060c99cf1a1f1548167e886fa2e1e75a01c67b55b95c9c7e8
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [188]:
from sklearn.metrics import accuracy_score

In [213]:
answers=[]
for smth in checks:
  answers.append(tonalizator(smth))

243
-106
163
-57
457
-161
534
-198
483
-210
300
-117
162
-66
696
-229
147
-75
424
-207


In [189]:
score = accuracy_score(answers, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [190]:
print(score)

0.0


Score получилась ужасная -- почему? Вероятно потому, что, несмотря на то, что были взяты банки отзывов одинакового размера, в банке с уникальными словами для негативных отзывов более чем в три раза больше слов, а значит, встретить их в отзывах более вероятно. Эта проблема решается значительным увеличением количества отзывов (что мне сделать не удалось из-за неудобства парсинга), либо изменением фильтра частоты (что будет сделано в коде чуть ниже). Также хочется отметить, что одной из возможных мулек может стать векторизация отзывов (впрочем, кажется, на таком маленьком количестве данных это все равно делать не особенно есть смысл).

In [199]:
goodfiltred2=[]
badfiltred2=[]
for smth in dictgood:
  if dictgood[smth]>6:
    goodfiltred2.append(smth)
for smth in dictbad:
  if dictbad[smth]>6:
    badfiltred2.append(smth)

In [200]:
def tonalizator2(rate):
  ton=0
  for word in rate:
    if word in goodfiltred2:
      ton+=1
    elif word in badfiltred2:
      ton=ton-1
  if ton>=0:
    return(1)
  else:
    return(0)

In [201]:
answers2=[]
for smth in checks:
  answers2.append(tonalizator2(smth))

это не помогает, тк негативных слов все еще в 4-5 раз больше. Может помочь следующее решение: отфильтровать списки слов и от каждого оставить n самых частотных

In [210]:
sorted_good = {}
sorted_keys = sorted(dictgood, key=dictgood.get, reverse=True) 

for w in sorted_keys:
    sorted_good[w] = dictgood[w]

top200good=list(sorted_good.keys())[1:201]

In [211]:
sorted_bad = {}
sorted_keys = sorted(dictbad, key=dictbad.get, reverse=True) 

for w in sorted_keys:
    sorted_bad[w] = dictbad[w]

top200bad=list(sorted_bad.keys())[1:201]

In [212]:
def tonalizator3(rate):
  ton=0
  for word in rate:
    if word in top200good:
      ton+=1
    elif word in top200bad:
      ton=ton-1
  if ton>=0:
    return(1)
  else:
    return(0)

In [215]:
answers=[]
for smth in checks:
  answers.append(tonalizator(smth))

все еще не работает, как должно, и все еще виню в этом нехватку отзывов. Соответственно, и какуюзнибудь векторизацию (которую в теории можно предложить как мульку, улучшающую качество), тоже сейчас использовать смысла нет.